## Imports

In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

In [ ]:
import numpy as np                                                              
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import FormatStrFormatter
import scipy
from skimage.measure import compare_psnr
import tensorflow as tf                                                         
import data.data_selector as ds                                                   
import analysis.analysis_picker as ap
import utils.plot_functions as pf                                               
import utils.data_processing as dp

## Model parameters

In [ ]:
fontsize = 12
figsize = (12, 12)

In [ ]:
class analysis_params(object):
  def __init__(self):
    self.model_type = "lca_subspace"
    self.model_name = "lca_subspace_vh"
    self.version = "2.0"
    self.save_info = "analysis_train_kurakin_targeted"
    self.overwrite_analysis_log = False

# Computed params
analysis_params = analysis_params()
analysis_params.project_dir = (os.path.expanduser("~")+"/Work/Projects/")
analysis_params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+analysis_params.model_name)

## Load analyzer

In [ ]:
analyzer = ap.get_analyzer(analysis_params.model_type)
analyzer.setup(analysis_params)
analyzer.setup_model(analyzer.model_params)
analyzer.load_analysis(save_info=analysis_params.save_info)
analyzer.model_name = analysis_params.model_name

## Group invariance reconstructions

In [ ]:
import itertools
def grid_angles(num_examples_per_dimension, num_neurons_per_group):
  angles = [np.linspace(0, np.pi, num_examples_per_dimension) for _ in range(num_neurons_per_group-2)]
  angles += [np.linspace(0, 2*np.pi, num_examples_per_dimension)]
  angles = [angle for angle in itertools.product(*angles)]
  return np.stack(angles, axis=0)

def random_angles(init, num_steps, step_size, momentum_weight=0.0):
  init = np.asarray(init)
  prev_step = 0.0
  angles = [init]
  for step in range(1, num_steps):
    delta = np.random.normal(0.0, 1.0, size=init.shape)
    delta_angle = momentum_weight * prev_step + step_size * delta
    new_angle = angles[step-1] + delta_angle
    for dim in range(new_angle.ndim-1):
      if new_angle[dim] > np.pi:
        new_angle[dim] = new_angle[dim] - np.pi
    if new_angle[-1] > 2*np.pi:
      new_angle[-1] = new_angle[-1] - (2 * np.pi)
    angles.append(new_angle)
  return np.stack(angles, axis=0)

def less_random_angles(init, steps_per_direction, num_directions, step_size):
  init = np.asarray(init)
  angles = [init]
  for direction in range(num_directions):
    delta = np.random.normal(0.0, 1.0, size=init.shape)
    for step in range(steps_per_direction):
      delta_angle = step_size * delta
      new_angle = angles[step-1] + delta_angle
      for dim in range(new_angle.ndim-1):
        if new_angle[dim] > np.pi:
          new_angle[dim] = new_angle[dim] - np.pi
      if new_angle[-1] > 2*np.pi:
        new_angle[-1] = new_angle[-1] - (2 *np.pi)
      angles.append(new_angle)
  return np.stack(angles, axis=0)

def angles_to_vectors(angles, target_group, num_groups, num_neurons_per_group):
  # https://en.wikipedia.org/wiki/N-sphere#Spherical_coordinates
  zs = []
  for angle in angles:
    z = np.zeros((num_groups, num_neurons_per_group))
    for group_neuron_idx in range(num_neurons_per_group):
      if group_neuron_idx == 0:
        z[target_group, group_neuron_idx] = np.cos(angle[group_neuron_idx])
      elif group_neuron_idx > 0 and group_neuron_idx <= num_neurons_per_group-2:
        prev_group_angles = [np.sin(angle[prev_group_neuron_idx])
          for prev_group_neuron_idx in range(group_neuron_idx)]
        z[target_group, group_neuron_idx] = np.prod(prev_group_angles)*np.cos(angle[group_neuron_idx])
      else: # group_neuron_idx == num_neurons_per_group - 1
        prev_group_angles = [np.sin(angle[prev_group_neuron_idx])
          for prev_group_neuron_idx in range(group_neuron_idx)]
        z[target_group, group_neuron_idx] = np.prod(prev_group_angles)
    zs.append(z)
  zs = np.stack(zs, axis=0)
  return zs

In [ ]:
target_group = 1
num_examples_per_dimension = 10
step_size = np.pi/10
momentum_weight = 2.0
num_directions = 100
steps_per_direction = 10
num_steps = num_directions * steps_per_direction

num_groups = analyzer.model_params.num_groups
num_neurons = analyzer.model_params.num_neurons
num_neurons_per_group = num_neurons // num_groups
init = [0]*num_neurons_per_group

# Traversal of the space one axis at a time
#angles = grid_angles(num_examples_per_dimension, num_neurons_per_group)

# Traversal of the space via random walk w/ momentum
#angles = random_angles(init, num_steps, step_size, momentum_weight)

# Traversal of the space via endpoint-finding
angles = less_random_angles(init, steps_per_direction, num_directions, step_size)

In [ ]:
target_groups = [1, 2, 5, 12, 66, 141, 154, 230, 305] # 35, 45, 81

In [ ]:
recons = []
for target_group in target_groups:
  zs = angles_to_vectors(angles, target_group, num_groups, num_neurons_per_group)
  sigmas = np.zeros((zs.shape[0], analyzer.model_params.num_groups))
  sigmas[:, target_group] = 1
  input_shape = [zs.shape[0]]+ analyzer.model.get_input_shape()[1:]
  feed_dict = analyzer.model.get_feed_dict(np.zeros(input_shape), is_test=True)
  feed_dict[analyzer.sigmas] = sigmas
  feed_dict[analyzer.zs] = zs
  recons.append(analyzer.evaluate_tf_tensor(analyzer.group_recons, feed_dict))

In [ ]:
num_plots_y = len(recons)
num_plots_x = len(recons)#len(angles)
fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(num_plots_y, num_plots_x+1, wspace=0.1, hspace=0.0)
for recon_ax in range(num_plots_y):
  y_ax = 0
  #for angle_ax in range(0, len(angles), len(angles)//num_plots_x):
  #for angle_ax in range(num_plots_x):
  for angle_ax in range(0, 3*num_plots_x, 3):
    ax = pf.clear_axis(fig.add_subplot(gs[recon_ax, y_ax]))
    img = recons[recon_ax][angle_ax].reshape([analyzer.model_params.patch_edge_size]*2)
    img = dp.normalize_data_with_max(img)[0]
    ax.imshow(img, cmap="Greys_r", vmin=-1, vmax=1)
    y_ax += 1
plt.show()

fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"group_invariance_tiled.png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
for recon_id in range(len(recons)):
  fig = plt.figure(figsize=figsize)
  ax = pf.clear_axis(fig.add_subplot())
  ims = []
  for i in range(recons[recon_id].shape[0]):
    recon = recons[recon_id][i].reshape([analyzer.model_params.patch_edge_size]*2)
    recon = dp.normalize_data_with_max(recon)[0]
    im = ax.imshow(recon, animated=True, cmap="Greys_r", vmin=-1, vmax=1)
    ims.append([im])
  ani = animation.ArtistAnimation(fig, ims, interval=50)
  ani.save(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_group_"+str(target_groups[recon_id])+"_recons.mp4")

### Full image recons

In [ ]:
normed_image = (
  (analyzer.full_image - np.min(analyzer.full_image))
  / (np.max(analyzer.full_image) - np.min(analyzer.full_image))).astype(np.float32)

normed_recon = (
  (analyzer.full_recon - np.min(analyzer.full_recon))
  / (np.max(analyzer.full_recon) - np.min(analyzer.full_recon))).astype(np.float32)

fig, ax = plt.subplots(1, 2, figsize=figsize)
ax[0] = pf.clear_axis(ax[0])
ax[0].imshow(np.squeeze(normed_image), cmap="Greys_r")
ax[0].set_title("Input Image", fontsize=fontsize)
ax[1] = pf.clear_axis(ax[1])
ax[1].imshow(np.squeeze(normed_recon), cmap="Greys_r")
percent_active = "{:.2f}".format(analyzer.recon_frac_act*100)
psnr = "{:.2f}".format(compare_psnr(normed_image, normed_recon, data_range=1))
ax[1].set_title("Reconstruction\n"+percent_active+" percent active"+"\n"+"PSNR = "+psnr, fontsize=fontsize)
plt.show()
fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_image_recon.png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)

### VanHateren

### Group coactivation stats

In [ ]:
data = ds.get_data(analyzer.model_params)
data = analyzer.model.preprocess_dataset(data, analyzer.model_params)
data = analyzer.model.reshape_dataset(data, analyzer.model_params)

In [ ]:
num_imgs = 10000#data["train"].images.shape[0]
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config, graph=analyzer.model.graph) as sess:
  feed_dict = analyzer.model.get_feed_dict(data["train"].images[0:num_imgs,...])
  sess.run(analyzer.model.init_op, feed_dict)
  analyzer.model.load_full_model(sess, analyzer.analysis_params.cp_loc)
  run_list = [analyzer.model.a, analyzer.model.module.group_activity, analyzer.model.module.group_angles]
  neuron_activations, group_activations, group_angles = sess.run(run_list, feed_dict)

In [ ]:
cent_group_activations = np.zeros_like(group_activations)
for group_id in range(analyzer.model.module.num_groups):
  cent_group_activations[:,group_id] = group_activations[:,group_id] - np.mean(group_activations[:,group_id])

In [ ]:
#cov = 1/num_imgs * np.dot(group_activations.T, group_activations)
cov = 1/num_imgs * np.dot(np.squeeze(cent_group_activations).T, np.squeeze(cent_group_activations))
np.fill_diagonal(cov, 0.0)

In [ ]:
fig = plt.figure(figsize=figsize)
ax = pf.clear_axis(fig.add_subplot())
im = ax.imshow(cov, cmap="Greys_r")
vmin = np.min(cov)
vmax = np.max(cov)
vmean = vmax - ((vmax - vmin) / 2)
pf.add_colorbar_to_im(im, aspect=20, ticks=[vmin, vmean, vmax], labelsize=fontsize)
ax.set_title("Covariance matrix for group activations", fontsize=fontsize)
plt.show()

fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_activity_covariance.png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
group_idx = 1
num_bins = 1000
indiv_group_act = group_activations[:, group_idx]
bins = np.linspace(np.min(indiv_group_act), np.max(indiv_group_act), num_bins)
hist, bin_edges = np.histogram(indiv_group_act.flatten(), bins)
bin_left, bin_right = bin_edges[:-1], bin_edges[1:]
bin_centers = bin_left + (bin_right - bin_left)/2

fig, ax = plt.subplots(1, figsize=figsize)
ax.bar(bin_centers, hist, width=2.0, log=True, align="center", color='k')
ax.set_xticks(bin_left, minor=True)
ax.set_xticks(bin_left[::10], minor=False)
ax.xaxis.set_major_formatter(FormatStrFormatter("%0.0f"))
ax.set_xlim([np.min(indiv_group_act), np.max(indiv_group_act)])
vmin = np.min(indiv_group_act)
vmax = np.max(indiv_group_act)
vmean = vmax - ((vmax - vmin)/2)
ax.set_xticks([vmin, vmean, vmax])
ax.set_title("Activity histogram of group "+str(group_idx)+" for "+str(len(indiv_group_act))+" images", fontsize=fontsize)
ax.set_xlabel("Activation", fontsize=fontsize)
ax.set_ylabel("Log Count", fontsize=fontsize)
ax.tick_params("both", labelsize=fontsize)
plt.show()
fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_group"+str(group_idx)+"_activity_hist.png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
num_images = 500
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
for neuron_id in range(neuron_activations.shape[1]):
  ax.scatter([neuron_id]*num_images, neuron_activations[:num_images, neuron_id], s=0.8, alpha=0.01, color='k')
ax.set_title("Individual Neuron Activity", fontsize=fontsize)
ax.set_xlabel("Neuron Index", fontsize=fontsize)
ax.set_ylabel("Activity", fontsize=fontsize)
ax.tick_params("both", labelsize=fontsize)
plt.show()

## Construct analysis plots

In [ ]:
fig = pf.plot_loc_freq_summary(analyzer.bf_stats, figsize=[figsize[0], figsize[1]/3], fontsize=fontsize)
fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_location_frequency_centers.png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
neuron_angles = analyzer.get_neuron_angles(analyzer.bf_stats)

In [ ]:
angle_hist_fig = pf.plot_weight_angle_histogram(neuron_angles[1], num_bins=50, angle_min=0, angle_max=180, figsize=(8,8))
angle_hist_fig.savefig(analyzer.analysis_out_dir+"/vis/neuron_angle_histogram.png")

In [ ]:
weights = analyzer.evals["lca_subspace/weights/w:0"]
weight_indices = np.stack([np.array(id_list) for id_list in analyzer.model.module.group_ids], axis=0)
pooling_weights = np.stack([weights[:, id_list] for id_list in analyzer.model.module.group_ids], axis=1)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits import axes_grid1
import utils.data_processing as dp
def plot_group_weights(weights, group_ids, title="", figsize=None,  save_filename=None):
  """
    weights: [np.ndarray] of shape [num_neurons, num_input_y, num_input_x]
    group_ids: [list of lists] containing ids for each group [[,]*neurons_per_group,]*num_groups
  """
  num_neurons = weights.shape[0]
  for weight_id in range(num_neurons):
    weights[weight_id,...] = weights[weight_id,...] - weights[weight_id,...].mean()
    weights[weight_id,...] = weights[weight_id,...] / (weights[weight_id,...].max()-weights[weight_id,...].min())
  vmin = np.min(weights)
  vmax = np.max(weights)
  indices = [idx for id_list in group_ids for idx in id_list]
  num_groups = len(group_ids)
  num_groups_x = int(np.floor(np.sqrt(num_groups)))
  num_groups_y = int(np.ceil(np.sqrt(num_groups)))
  num_neurons_per_group = len(group_ids[0])
  num_neurons_x = int(np.floor(np.sqrt(num_neurons_per_group)))
  num_neurons_y = int(np.ceil(np.sqrt(num_neurons_per_group)))
  outer_spacing = 0.20
  inner_spacing = 0.1
  fig = plt.figure(figsize=figsize)
  gs1 = gridspec.GridSpec(num_groups_y, num_groups_x,
    hspace=outer_spacing*num_groups_y/(num_groups_x+num_groups_y),
    wspace=outer_spacing*num_groups_x/(num_groups_x+num_groups_y))
  neuron_index = 0
  for group_plot_id in np.ndindex((num_groups_y, num_groups_x)):
    gs_inner = gridspec.GridSpecFromSubplotSpec(num_neurons_y, num_neurons_x, gs1[group_plot_id],
      hspace=inner_spacing*num_neurons_y/(num_neurons_x+num_neurons_y),
      wspace=inner_spacing*num_neurons_x/(num_neurons_x+num_neurons_y))
    for inner_plot_id in np.ndindex((num_neurons_y, num_neurons_x)):
      ax = pf.clear_axis(fig.add_subplot(gs_inner[inner_plot_id]))
      ax.set_aspect("equal")
      if neuron_index < num_neurons:
        ax.imshow(weights[indices[neuron_index], ...], cmap="Greys_r", vmin=vmin, vmax=vmax)
        neuron_index += 1
  fig.suptitle(title, y=0.9, x=0.5, fontsize=20)
  if save_filename is not None:
    fig.savefig(save_filename)
    plt.close(fig)
    return None
  plt.show()
  return fig

In [ ]:
weights = np.reshape(analyzer.evals["lca_subspace/weights/w:0"].T, [analyzer.model.params.num_neurons,
      int(np.sqrt(analyzer.model.params.num_pixels)), int(np.sqrt(analyzer.model.params.num_pixels))])
weight_fig = plot_group_weights(np.squeeze(weights), analyzer.model.module.group_ids,
  title="Dictionary", figsize=(18,18))
weight_fig.savefig(analyzer.analysis_out_dir+"/vis/"+"group_phi.png")

### MNIST